Neste notebook, vamos codificar Redes Neurais de forma manual para tentar entender intuitivamente como elas são implementadas na prática.

# Sumário

- [Exemplo 1](#Exemplo-1)
- [Exemplo 2](#Exemplo-2)
- [O que precisamos para implementar uma Rede Neural?](#O-que-precisamos-para-implementar-uma-Rede-Neural?)
- [Referências](#Referências)

# Imports e Configurações

In [1]:
import numpy as np

# Exemplo 1

<img src="./images/backprop_example_1.png">

In [4]:
# implementando a função de ativação sigmoid
def sigmoid(x, derivative=False):
    # Função de ativação sigmoid
    if derivative:
        # Calcula a derivada da função sigmoid em relação a x
        # y é o resultado da função sigmoid aplicada a x
        # Retorna a derivada da função sigmoid aplicada a x
        y = sigmoid(x)
        return y * (1 - y)
        
    # Retorna o resultado da função sigmoid aplicada a x
    return 1 / (1 + np.exp(-x))

In [6]:
# inicializando entradas
x = np.array([[0.05, 0.10]])
y = np.array([[0.01, 0.99]])

# inicializando pesos e bias
w1 = np.array([[0.15, 0.20], [0.25, 0.30]]) # array com 2x2
b1 = np.array([[0.35]])

# pesos e bias da segunda camada
w2 = np.array([[0.40, 0.45], [0.50, 0.55]]) # array 2x2
b2 = np.array([[0.60]])

learning_rate = 0.5

In [17]:
# camada feed foward
for i in range(1):
    # primeira camada
    inp1 = np.dot(x, w1.T) + b1
    h1 = sigmoid(inp1)
    
    # segunda camada
    inp2 = np.dot(h1, w2.T) + b2
    out = sigmoid(inp2)
    
    # custo
    cost = 0.5 * np.sum((y - out) ** 2)
    
    # backpropagation
    dout = -(y - out) # derivada do custo (antes usado como 1 nos exemplos)
    
    # segunda camada
    dinp2 = sigmoid(inp2, derivative=True) * dout
    dh1 = np.dot(dinp2, w2)
    dw2 = np.dot(dinp2.T, h1)
    db2 = 1 * dinp2.sum(axis=0, keepdims=True) # multiplicando mantendo dimensao para manter como um vetor 2d
    
    # primeira camada
    dinp1 = sigmoid(inp1, derivative = True) * dh1
    dx = np.dot(dinp1, w1)
    dw1 = np.dot(dinp1.T, x)
    db1 = 1 * dinp1.sum(axis=0, keepdims = True)

    # atualizando os pesos e bias
    w2 = w2 - learning_rate * dw2
    b2 = b2 - learning_rate * db2
    w1 = w1 - learning_rate * dw1
    b1 = b1 - learning_rate * db1

# Exemplo 2

In [19]:
# def linear(x, derivative=False):
#     return np.ones_like(x) if derivative else x

# def relu(x, derivative=False):
#     if derivative:
#         x = np.where(x <= 0, 0, 1)
#     return np.maximum(0, x)

# def softmax(x, y_oh=None, derivative=False):
#     if derivative:
#         y_pred = softmax(x)
#         k = np.nonzero(y_pred * y_oh)
#         pk = y_pred[k]
#         y_pred[k] = pk * (1.0 - pk)
#         return y_pred
    
#     exp = np.exp(x)    
#     return exp / np.sum(exp, axis=1, keepdims=True) # mantendo as dimensoes para somar os valores linha a linha mantendo as dimensoes

# def neg_log_likelihood(y_oh, y_pred, derivative=False):
#     k = np.nonzero(y_pred * y_oh)
#     pk = y_pred[k]
#     if derivative:
#         y_pred[k] = (-1.0 / pk)
#         return y_pred
#     return np.mean(-np.log(pk))

# def softmax_neg_log_likelihood(y_oh, y_pred, derivative=False):
#     y_softmax = softmax(y_pred)
#     if derivative:
#         k = np.nonzero(y_pred * y_oh)
#         dlog = neg_log_likelihood(y_oh, y_softmax, derivative=True)
#         dsoftmax = softmax(y_pred, y_oh, derivative=True)
#         y_softmax[k] = dlog[k] * dsoftmax[k]
#         return y_softmax / y_softmax.shape[0]
#     return neg_log_likelihood(y_oh, y_softmax)

import numpy as np

def linear(x, derivative=False):
    # Retorna uma matriz de mesma forma que x com todos os elementos iguais a 1
    if derivative:
        return np.ones_like(x)
    else:
        return x

def relu(x, derivative=False):
    # Função de ativação ReLU (Rectified Linear Unit)
    if derivative:
        # Retorna uma matriz de mesma forma que x com valores 0 onde x é menor
        #ou igual a 0 e 1 onde x é maior que 0
        return np.where(x <= 0, 0, 1)
    else:
        # Retorna uma matriz de mesma forma que x com valores 0 onde x é menor que
        #0 e x onde x é maior ou igual a 0
        return np.maximum(0, x)

def softmax(x, y_oh=None, derivative=False):
    # Função de ativação Softmax
    if derivative:
        # Calcula a derivada da função softmax em relação a x
        # y_oh é a codificação one-hot do vetor de rótulos verdadeiros y
        # Retorna uma matriz com as derivadas parciais da função softmax em relação a cada elemento de x
        y_pred = softmax(x)
        k = np.nonzero(y_pred * y_oh)
        pk = y_pred[k]
        y_pred[k] = pk * (1.0 - pk)
        return y_pred
    
    exp = np.exp(x)    
    # Retorna uma matriz com as probabilidades resultantes da função softmax aplicada a cada linha de x
    return exp / np.sum(exp, axis=1, keepdims=True)

def neg_log_likelihood(y_oh, y_pred, derivative=False):
    # Calcula a perda negativa do logaritmo de verossimilhança
    # y_oh é a codificação one-hot do vetor de rótulos verdadeiros y
    # y_pred é o vetor de probabilidades previstas pelo modelo
    k = np.nonzero(y_pred * y_oh)
    pk = y_pred[k]
    if derivative:
        # Calcula a derivada da perda em relação a y_pred
        # Retorna uma matriz com as derivadas parciais da perda em relação a cada elemento de y_pred
        y_pred[k] = (-1.0 / pk)
        return y_pred
    # Retorna a média da perda negativa do logaritmo de verossimilhança
    return np.mean(-np.log(pk))

def softmax_neg_log_likelihood(y_oh, y_pred, derivative=False):
    # Calcula a perda negativa do logaritmo de verossimilhança combinada com a função softmax
    # y_oh é a codificação one-hot do vetor de rótulos verdadeiros y
    # y_pred é o vetor de probabilidades previstas pelo modelo
    y_softmax = softmax(y_pred)
    if derivative:
        # Calcula a derivada da perda combinada em relação a y_pred
        # Retorna uma matriz com as derivadas parciais da perda combinada em 
        #relação a cada elemento de y_pred
        k = np.nonzero(y_pred * y_oh)
        dlog = neg_log_likelihood(y_oh, y_softmax, derivative=True)
        dsoftmax = softmax(y_pred, y_oh, derivative=True)
        y_softmax[k] = dlog[k] * dsoftmax[k]
    # Retorna a perda negativa do logaritmo
    return y_softmax / y_softmax.shape[0]

In [3]:
x = np.array([[0.1, 0.2, 0.7]])
y = np.array([[1, 0, 0]])
w1 = np.array([[0.1, 0.2, 0.3], [0.3, 0.2, 0.7], [0.4, 0.3, 0.9]])
b1 = np.ones((1,3))
w2 = np.array([[0.2, 0.3, 0.5], [0.3, 0.5, 0.7], [0.6, 0.4, 0.8]])
b2 = np.ones((1,3))
w3 = np.array([[0.1, 0.4, 0.8], [0.3, 0.7, 0.2], [0.5, 0.2, 0.9]])
b3 = np.ones((1,3))

learning_rate = 0.01

for i in range(301):
    # feedforward
    # 1a camada
    

    
    # 2a camada

    
    # 3a camada

    
    # backpropagation
    # insira seu código aqui!
    

for w in [w1, w2, w3]:
    print(w)

IndentationError: expected an indented block after 'for' statement on line 12 (977196323.py, line 27)

# O que precisamos para implementar uma Rede Neural?

# Referências

- [Neural Network from Scratch](https://beckernick.github.io/neural-network-scratch/)
- [Backpropagation Algorithm](https://theclevermachine.wordpress.com/tag/backpropagation-algorithm/)
- [Back-Propagation is very simple. Who made it Complicated ?](https://becominghuman.ai/back-propagation-is-very-simple-who-made-it-complicated-97b794c97e5c)
- [A Step by Step Backpropagation Example](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/)
- [Understanding softmax and the negative log-likelihood](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/)